# Inverse Kinematics and PID control
## Yu Jun Shen
In this homework assignemnt, you will be tasked with creating an IK solver and controlling a robot arm using PID control (specifically the Franka Panda 7DoF robot). We will be creating two versions of the controller, one that uses pure velocity control and another that using torque control with feedforward dynamics. 

### Import mujoco and numpy

In [37]:
import mujoco
import mujoco_viewer
import numpy as np
import scipy as sp

from autograder import autograder
from sim import Sim

franka_velocity_ctrl_sim = Sim(fname='./franka_emika_panda/scene_velocity_actuators.xml')
franka_torque_ctrl_sim = Sim(fname='./franka_emika_panda/scene_torque_actuators.xml')


### Robot Info: Franka Emika Panda Robot
We will be importing the Franka Panda robot. Take a look at the robot tree in `franka_emika_panda/panda.xml` file. Rather than importing the robot, we will instead import a specific `scene_X.xml` file that specifies different joint actuation on the robot. Note that the `panda.xml` only specifies the robot body and joints. There are two other files that specify the type of actuation on the robot (`vel_act.xml` and `torque_act.xml`) which we will use to demonstrate the different levels of robot control. 

### Assignment Questions 
Go through this notebook and answer all the questions below. There are autograder helpers which will tell you whether you have good code or not. Please use those to ensure that you can safely move on the next problem. This will prevent any bugs from being carried over to the next problem.

## Rotation matrices and error signal
Before we begin, you need to write up a few functions that will help you compute errors on rotation matrices that you will use to control the robot. First, we will write two helper functions `hat` and `unhat` which converts vectors in $\mathbb{R}^3 \to \text{se}(3)$ and back. This will be used when you compute the distance between two rotation matrices. Specifically, the error between two rotation matrices can be defined using the axis-angle formulation when we discussed exponential coordinates. Recall that $R = e^{\hat{\omega}\theta}$ which is a rotation in some axis $\omega$ by the amount of $\theta$. We can convert $R$ into its components $\hat{\omega} \theta \in \mathbb{R}^3$ by using the matrix logarithm $\text{logm}$. We will then need to convert the resulting matrix back into $\mathbb{R}^3$ using the `unhat` operator. We will be using the scipy library to compute `unhat(sp.linalg.logm(R)) = w` where here `w` is combined with $\theta$. To compute the error between two rotation matrices, we simply multiply the two $R_1 R_2^\top$. For example, if $R_1 = R_{WD}$ and $R_2 = R_{WEE}$, then we want an $R$ such that $R_{WD} =R R_{WEE}$. Therefore, $R = R_{WD} R_{WEE}^\top$ is our relative rotation transform.


### Q1 a. Hat, Unhat 
Write a function `hat` that converts a vector $w\in\mathbb{R}^3 \to \text{se}(3)$ and another function `unhat` that goes the opposite direction.

In [38]:
def hat(w):
    """ By definition of hat operator in notes 1, eq. 1.32 """
    w_hat = np.array([[0    , -w[2],  w[1]],
                      [w[2] , 0    , -w[0]],
                      [-w[1],  w[0],    0]])
    return w_hat
 
def unhat(R):
    """ By observation of the matrix in 1.32 """
    return np.array([-R[1][2], R[0][2], -R[0][1]])

In [39]:
autograder.testQ1a(hat, unhat)

Passed Debug test


### Q1 b. Rotation error
Write a function that computes the error between two rotation matrices $R_1, R_2$.

In [40]:
def rot_err(R1, R2):
    """ 
    Using numpy transpose and matrix multiplication terms, 
    compute relative rotation transform and then get w per formula
    Goes from R2 to R1
    """
    R2_transpose = np.transpose(R2)
    R_rel = np.matmul(R1, R2_transpose)
    
    #print(unhat(sp.linalg.logm(R_rel)))    # outputs 1x3 vector

    return unhat(sp.linalg.logm(R_rel))

In [41]:
autograder.testQ1b(rot_err)

Passed Debug test


## Inverse Kinematics Solver
We will now create the IK solver. We will use some helper functions to get the rotation, position, and jacobian of a site in the `.xml` file. Look through the files and find the tags for `site`. Specifically, look for the two tags `ee_site` and `target_site` within the `worldbody` tags. These will be the two target frames on a the robot and in the world respectively. Understand the functions below and answer the subsequent questions. 

In [42]:
def getSitePose(model, data, site_name):
    """
        This function takes in the robot model, data, and the site name of the reference frame. 
        The output of the function returns a tuple of a rotation matrix R and a position p 
        relative to the world frame. 
    """
    # mujoco specific function
    site_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, site_name)
    # data structure for obtaining the position and orientation of a site reference frame
    # note that rotation matrices are formated as a 9-dim vector and need to be reshaped
    p = data.site_xpos[site_id]
    R = data.site_xmat[site_id].reshape((3,3))
    return (R, p)

def getSiteJacobian(model, data, site_name):
    """
        This function takes in the robot model, data, and the site name of the reference frame. 
        The output is the total jacobian matrix. Note that mujoco gives jacobian 
        position and rotation matrices which need to be stacked together.
    """
    body_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, site_name)
    jacp = np.zeros((3, model.nv))
    jacr = np.zeros((3, model.nv))
    mujoco.mj_jacSite(model, data, jacp, jacr, body_id)
    return np.concatenate([jacp, jacr], axis=0)


In [43]:
# From panda.xml file
# <site name="ee_site" pos="0 0 0.1" euler="0 0 -1.57079" size="0.05 0.05 0.05" type="sphere" rgba="0 0 0.5 0.5"/>


## Q2. Write the IK step. 
Write function to compute $\Delta q$, for $q_{k+1} = q_k + \Delta q$, where $\Delta q$ is the solution to the minimum joint deviation based on the Taylor expansion of $f(q) - x_\text{des}$, and $f(q)$ is your forward kinematics (assume that this is given as $x_\text{curr}$).

In [44]:
# dev testing concatenate
# x = np.array([1,2,3])
# y = np.array([2,3,4])
# z = np.concatenate((x,y), axis=None)
# print(z)

In [45]:
def calcIKStep(jac, x_des, x_curr):
    """
        This function takes in the jacobian matrix of the x_\text{curr} = f(q) forward kinematics, 
        and the desired frame rotation matrix and position. 
        Assumes x_des, x_curr = (R, p)
        Output of this function computes $\Delta q$ using the jacobian.
        
        Based on eq. 1.6 of notes 2
    """
    # break up subtraction of R and p separately
    rot_error = rot_err(x_des[0], x_curr[0])    # rotation error between matrices 
    pos_error = x_des[1] - x_curr[1]
    cur_error = np.concatenate((pos_error, rot_error), axis=None)
    delta_q = np.matmul(np.linalg.pinv(jac), cur_error)
    
    return delta_q

def IKSolve(q, jac, x_des, x_curr):
    dq = calcIKStep(jac, x_des, x_curr)
    return q + dq


# Control!
We will now implement different control strategies to move our robot. We will use the function below to simulate the control method used. Make sure you run the cell and load the function. Note that for the following examples you need to calculate the error from the poses of the reference frames.

In [46]:
def simulateCtrl(sim, ctrl_method, render=True, t_max=1000):
    sim.data.qpos = sim.model.key_qpos
    mujoco.mj_step(sim.model, sim.data)
    log = []
    if render:
        viewer = mujoco_viewer.MujocoViewer(sim.model, sim.data)
        viewer.vopt.frame = 3
    for t in range(t_max):
        u = ctrl_method(sim.model, sim.data)
        sim.data.ctrl = u 
        mujoco.mj_step(sim.model, sim.data)
        if render:
            if viewer.is_alive:
                viewer.render()
    if render:
        viewer.close()

## Q3 Velocity control
Create a controller that returns $\Delta q$ given the end-effector pose and the target site pose. Using the `calcIKStep` to compute $\Delta q$. Run the `simulateCtrl` using the `franka_velocity_ctrl_sim` simulation. Describe what you see. 

## Response:

Under velocity control, the robot arm moves towards the position directly but slumps over due to the effect of weight (not taken into consideration). Thus the end effector is near but not directly at the desired position and orientation. 

In [47]:
def velocity_ctrl(model, data):
    
    R_ee, p_ee = getSitePose(model, data, "ee_site") # rotation matrix R and position p 
    R_ts, p_ts = getSitePose(model, data, "target_site") 
    jacobian = getSiteJacobian(model, data, "ee_site")  # Jacobian stacked 
    # print(jacobian)  # 6 by 7 matrix, since robot has seven joints
    
    x_curr = (R_ee, p_ee)
    x_des  = (R_ts, p_ts)
   
    solved_step = calcIKStep(jacobian, x_des, x_curr)   # q from data.qpos
    
    return solved_step

In [48]:
simulateCtrl(franka_velocity_ctrl_sim, velocity_ctrl)

## Q4 Torque control
Create a  basic PD controller that calculates torque values using $q$ from the `IKSolve` given the end-effector pose and the target site pose. Tune the PD values as best as you can.Describe what you see. 

## Response:

The robot arm moves towards the target position but also slumps over slightly, due to the effect of weight. By not specifying the mass matrix, we are effectively assigning it the identity matrix so the robot arm is also not moving in the most expedient direction. Increasing kp allows the end effector to move closer towards the target position and hold its position (drop less), but kd cannot be too large (otherwise the solver cannot converge). 

In [49]:
def torque_ctrl(model, data):
    """
    Output will be a 7 dim vec tau = kp (qdes - qpos) + kd * (-qvel)
    where data.qpos = qpos and data.qvel = qvel are from the robot 
    """ 
    
    kp = 200
    kd = 100
    
    # inverse kinematics for where to go
    R_ee, p_ee = getSitePose(model, data, "ee_site") # rotation matrix R and position p 
    R_ts, p_ts = getSitePose(model, data, "target_site") 
    jacobian = getSiteJacobian(model, data, "ee_site")  # Jacobian stacked 
    x_curr = (R_ee, p_ee)
    x_des  = (R_ts, p_ts)
    des_pos = IKSolve(data.qpos, jacobian, x_des, x_curr)   # q from data.qpos
    des_vel = 0
    
    # PD controller to implement torque
    # data.qpos for current position, data.qvel for current velocity
    current_pos = data.qpos
    current_vel = data.qvel
    
    q_des = kp*(des_pos - current_pos) + kd*( - current_vel)

    return q_des


In [50]:
simulateCtrl(franka_torque_ctrl_sim, torque_ctrl)

## Q4 Dynamic Comp. Torque control
Create a feed forward dynamic compensation controller that calculate target $q$ from the `IKSolve` given the end-effector pose and the target site pose. Tune the PD values as best as you can. Using the provided functions for obtaining the mass matrix and dynamic bias terms. Describe what you see. 

## Response:

Taking into account the real mass distribution, the arm now delivers the end effector accurately to the desired target position and orientation. The values of kp and kd tuned are much lower than the previous part because the weight is now being accounted for, so the control output is more tailored. Furthermore, increasing kp tends to create more overshoot as the robot arm swings but is faster, so kd needs to be adjusted to compensate. 

In [51]:
def getMassMatrix(model, data):
    mass_matrix = np.zeros((model.nv, model.nv))
    mujoco.mj_fullM(model, mass_matrix, data.qM)
    return mass_matrix

def getBiasTerms(model, data):
    return data.qfrc_bias

def feedforward_torque_ctrl(model, data):
    kp = 10
    kd = 2
    
    # inverse kinematics for where to go
    R_ee, p_ee = getSitePose(model, data, "ee_site") # rotation matrix R and position p 
    R_ts, p_ts = getSitePose(model, data, "target_site") 
    jacobian = getSiteJacobian(model, data, "ee_site")  # Jacobian stacked 
    x_curr = (R_ee, p_ee)
    x_des  = (R_ts, p_ts)
    des_pos = IKSolve(data.qpos, jacobian, x_des, x_curr)   # q from data.qpos
    des_vel = 0
    
    # PD controller to implement torque
    # data.qpos for current position, data.qvel for current velocity
    current_pos = data.qpos
    current_vel = data.qvel
    qacc_des = kp*(des_pos-current_pos) + kd*(des_vel-current_vel)
    
    bias_term = getBiasTerms(model, data)
    mass_matrix = getMassMatrix(model, data)
    
    torque = np.matmul(mass_matrix, qacc_des)+bias_term

    return torque
    

In [52]:
simulateCtrl(franka_torque_ctrl_sim, feedforward_torque_ctrl)